# KerasSurgeon must be installed in order to delete channels

In [ ]:
from kerassurgeon import Surgeon
from kerassurgeon import identify
from kerassurgeon import utils

from kerassurgeon.operations import delete_channels

import numpy as np

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

####################################################"

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, MaxPool2D 
from keras import optimizers
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers
#####################################################"
# load the model
from tensorflow.keras.applications.vgg16 import VGG16   

import tensorflow.keras.layers

model = Sequential()
weight_decay = 0.0005


model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", name="conv2d_13"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=1000, activation="softmax"))

print("Model created")

model.compile(loss='categorical_crossentropy', optimizer='sgd',
              metrics=['accuracy'])
print("Model compile")

VGGmodel = VGG16(weights='imagenet', include_top=True)
model.set_weights(VGGmodel.get_weights())
model.summary()

Using TensorFlow backend.


Model created



Model compile
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

img_path = 'Gazelle/Gazelle.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])
 
orig_acc = decode_predictions(preds, top=3)[0][0][2]
print('Prob:', orig_acc)


# This part to compute the difference in prediction probability when deleting each filter for the last convolution layer (conv2d_13)

In [ ]:
arryOfDictFiltByLayer = []
weights_dic = {}
for j in range (512):
    print ('filt{}'.format (j))
    filt = j
    layer = model.get_layer(index=16) ## conv13
    channel = [j] 
    new_model = delete_channels (model, layer, channel, copy=True)
    new_model.compile (optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
    preds = new_model.predict(x)
    New_accu = decode_predictions(preds, top=3)[0][0][2]
    print ('acc after purning (%.2f%%)', New_accu)
    print('difference: ', (orig_acc - New_accu))
    tuple = orig_acc - New_accu, 1
    weights_dic[filt] = tuple
    
    K.clear_session ()

    model = Sequential()
    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu", name="conv2_2"))

    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu", name="conv2_4"))

    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", name="conv2_7"))

    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", name="conv2_10"))

    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu", name="conv2_13"))

    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), name="last_max_pool"))
    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=1000, activation="softmax", name="predictions"))

    model.compile (optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print("Model created")

    VGGmodel = VGG16(weights='imagenet', include_top=True)
    model.set_weights(VGGmodel.get_weights())

    model.compile (optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
weights_dic_sort = sorted (weights_dic.items (), key=lambda kv: kv[1], reverse=True)
print ('Filters Importance {}\n'.format (1), weights_dic_sort)
arryOfDictFiltByLayer.append(weights_dic_sort)


filt0
Deleting 1/512 channels from layer: conv2d_13
acc after purning (%.2f%%) 0.989135
difference:  -7.241964e-05
Model created
filt1
Deleting 1/512 channels from layer: conv2_13
acc after purning (%.2f%%) 0.9892216
difference:  -0.00015896559
Model created
filt2
Deleting 1/512 channels from layer: conv2_13
acc after purning (%.2f%%) 0.98821104
difference:  0.00085157156
Model created
filt3
Deleting 1/512 channels from layer: conv2_13


acc after purning (%.2f%%) 0.98922604
difference:  -0.00016343594
Model created
filt4
Deleting 1/512 channels from layer: conv2_13


acc after purning (%.2f%%) 0.9893995
difference:  -0.00033688545
Model created
filt5
Deleting 1/512 channels from layer: conv2_13


acc after purning (%.2f%%) 0.9889904
difference:  7.2181225e-05
Model created
filt6
Deleting 1/512 channels from layer: conv2_13


acc after purning (%.2f%%) 0.98854524
difference:  0.0005173683
Model created
filt7
Deleting 1/512 channels from layer: conv2_13


...

acc after purning (%.2f%%) 0.9890676
difference:  -5.00679e-06
Model created
filt508
Deleting 1/512 channels from layer: conv2_13


acc after purning (%.2f%%) 0.9903784
difference:  -0.0013157725
Model created
filt509
Deleting 1/512 channels from layer: conv2_13


acc after purning (%.2f%%) 0.9890982
difference:  -3.5583973e-05
Model created
filt510
Deleting 1/512 channels from layer: conv2_13


acc after purning (%.2f%%) 0.9878171
difference:  0.0012454987
Model created
filt511
Deleting 1/512 channels from layer: conv2_13


acc after purning (%.2f%%) 0.9900365
difference:  -0.0009738803
Model created
Resp and loss for conv layer 1
 [(147, (0.5048464, 1)), (407, (0.075149655, 1)), (136, (0.030532181, 1)), (87, (0.025048256, 1)), (35, (0.01881349, 1)), (243, (0.014434576, 1)), (155, (0.012325823, 1)), (354, (0.012112558, 1)), (398, (0.011274338, 1)), (245, (0.010389924, 1)), (249, (0.010107994, 1)), (11, (0.010098338, 1)), (487, (0.009231985, 1)), (44, (0.008374274, 1)), (53, (0.008343875, 1)), (456, (0.005669117, 1)), (248, (0.005545199, 1)), (161, (0.0051465034, 1)), (52, (0.0042666793, 1)), (231, (0.003749609, 1)), (283, (0.0035712123, 1)), (125, (0.0032886863, 1)), (257, (0.0030542612, 1)), (227, (0.0029067993, 1)), (378, (0.002747357, 1)), (181, (0.002413869, 1)), (24, (0.0023272634, 1)), (418, (0.0023247004, 1)), (162, (0.0022107363, 1)), (443, (0.0020833611, 1)), (180, (0.0018445253, 1)), (144, (0.0018094778, 1)), (463, (0.0017859936, 1)), (22, (0.0016723275, 1)), (389, (0.0016081929, 1)), (506, (0.0